<a href="https://colab.research.google.com/github/nikhilagrawal2000/cerebro_melodious_medley/blob/master/cerebro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [25]:
! git clone https://github.com/nikhilagrawal2000/cerebro_melodious_medley.git
%cd cerebro_melodious_medley
! ls

Cloning into 'cerebro_melodious_medley'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 1), reused 4 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.
/content/cerebro_melodious_medley/cerebro_melodious_medley
extra_infos.json  README.md  test.csv  train.csv


In [0]:
df_training = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

labels = df_training.loc[:, 'is_listened']
ids = df_training.loc[:, 'ID']
ts_listen_df = df_training.loc[:, 'ts_listen']
release_data_df = df_training.loc[:, 'release_date']
#df_training = df_training.drop(['release_date'], axis =1)
df_training = df_training.drop(['is_listened'], axis = 1)
df_training = df_training.drop(['ID'], axis = 1)
df_training = df_training.drop(['ts_listen'], axis = 1)

In [0]:
ids_test = df_test.loc[:, 'ID']
ts_listen_df_test = df_test.loc[:, 'ts_listen']
release_data_df_test = df_test.loc[:, 'release_date']
#df_test = df_test.drop(['release_date'], axis =1)
df_test = df_test.drop(['ID'], axis = 1)
df_test = df_test.drop(['ts_listen'], axis = 1)

In [0]:
X_all = df_training
y_ = labels

In [0]:
for head in X_all:
    series = X_all[head]
    mean = series.mean()
    maximum = series.max()
    minimum = series.min()
    series = (series - minimum)/(maximum - minimum)
    X_all[head] = series

for head in df_test:
    series = df_test[head]
    mean = series.mean()
    maximum = series.max()
    minimum = series.min()
    series = (series - minimum)/(maximum - minimum)
    df_test[head] = series

In [30]:
n_rows = X_all.shape[0]
n_cols = X_all.shape[1]
print("Total number of rows: {} ".format(n_rows))
print("Total number of cols: {} ".format(n_cols))

Total number of rows: 632284 
Total number of cols: 13 


In [31]:
num_test = 120000
num_train = n_rows - num_test

from sklearn import cross_validation

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_, test_size=num_test)
print("Training set: {} samples".format(X_train.shape[0]))
print ("Test set: {} samples".format(X_test.shape[0]))

Training set: 512284 samples
Test set: 120000 samples


In [45]:
import time

def train_classifier(clf, X_train, y_train):
    print("Training {}...".format(clf.__class__.__name__))
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print ("Done!\nTraining time (secs): {:.3f}".format(end - start))

from sklearn import ensemble
from sklearn import tree
from sklearn.svm import SVC

#clf = ensemble.RandomForestClassifier()
#clf = SVC(gamma='auto')
#clf = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(max_depth = 5),n_estimators = 50, learning_rate = 1.0, algorithm = "SAMME.R")
clf = ensemble.GradientBoostingClassifier(n_estimators = 100, learning_rate =1, max_depth =5)
# Fit model to training data
train_classifier(clf, X_train, y_train)


Training GradientBoostingClassifier...
Done!
Training time (secs): 155.210


In [47]:
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print("Predicting labels using {}...".format(clf.__class__.__name__))
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print( "Done!\nPrediction time (secs): {:.3f}".format(end - start))
    return f1_score(target.values, y_pred, pos_label=1)

train_f1_score = predict_labels(clf, X_train, y_train)
print ("F1 score for training set: {}".format(train_f1_score))

Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 1.211
F1 score for training set: 0.8104504181447946


In [48]:
print("F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)))

predict_leader = clf.predict(df_test)

data = {'is_listened':predict_leader}

print(data)

predict_leader = pd.DataFrame.from_dict(data)

LeaderBoardFinal = pd.concat((ids_test,predict_leader),axis =1)

print(LeaderBoardFinal.shape)

LeaderBoardFinal.to_csv('submission.csv')

Predicting labels using GradientBoostingClassifier...
Done!
Prediction time (secs): 0.260
F1 score for test set: 0.8012475191380777
{'is_listened': array([1, 1, 1, ..., 0, 1, 1])}
(421542, 2)


In [0]:
#import xgboost as xgb
#from sklearn.metrics import accuracy_score

#model = xgb.XGBClassifier()
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)
#accuracy = accuracy_score(y_test, y_pred)
#print(accuracy)
